In [1]:
import json
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

In [32]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon")
%run data_prep_10step_5horizon.ipynb
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon")
%run LSTM_Model_Classes_no_embed.ipynb
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/")
%run Useful_Functions.ipynb

In [50]:
batch_size = 1

lookback = 1

input_size = 1
hidden_size = 64
num_layers = 2
output_size = 1  # Change per number of steps you want to predict out
horizon = 1  # Change per number of steps you want to predict out

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

train_dataset = TimeSeriesDataset(df2_70, lookback=lookback)
test_dataset = TimeSeriesDataset(df2_70_remain, lookback=lookback)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
len(test_dataset)

2991

# No CUDA

In [52]:
# Training loop
num_epochs = 500
patience = 50  # Number of epochs to wait for improvement

# Define loss function and optimizer
criterion = nn.MSELoss()
#criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_loss = float('inf')
best_model_state = None
epochs_without_improvement = 0

for epoch in range(num_epochs):
    model.train()

    train_loss = 0.0
    print(f"Epoch: {epoch}")

    for batch in train_loader: #for hero_ids, X, y in train_loader:

        hero_ids, X, y = batch #outputs = model(hero_ids, X)
        #print("Training Loop X Type", type(X))
        #print("Training Loop X shape", len(X))
        #print("Training Loop X", X)

        #print("Training Loop Heros Type", type(hero_ids))
        #print("Training Loop Heros shape", len(hero_ids))
        #print("Training Loop Heros", hero_ids)

        X = X[0]
        # print("Training Loop X Type", type(X))
        #print("Training Loop X shape", X.size())
        # print("Training Loop X", X)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch)
        #print("Model Outputs: ", outputs)
        #print("Model Outputs Size: ", outputs.size())

        y = y[0]
        # print("Training Loop Y Type", type(y))
        #print("Training Loop Y shape", y.size())
        #print("Training Loop Y", y)

        targets = y[:, -horizon:]  # Assuming you want to predict the last value of each time series

        #print("targets last size: ", targets.size())
        #print("targets last : ", targets)
        loss = criterion(outputs.squeeze(), targets.squeeze())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X.size(0)

    train_loss /= len(train_dataset)

    model.eval()
    test_loss = 0.0

    with torch.no_grad():
         for batch in test_loader: #for hero_ids, X, y in test_loader:
            # Forward pass
            hero_ids, X, y = batch
            X = X[0]
            y = y[0]
            outputs = model(batch) # outputs = model(hero_ids, X)

            targets = y[:, -horizon:]  # Assuming you want to predict the last value of each time series

            #print(outputs.shape)
            #print(targets.shape)
            loss = criterion(outputs.squeeze(), targets.squeeze())

            test_loss += loss.item() * X.size(0)
            #print(f"Test Loss for Batch: ", test_loss)

    test_loss /= len(test_dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_model_state = model.state_dict()
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"Early stopping at epoch {epoch+1}. Best test loss: {best_test_loss:.4f}")
        break


Epoch: 0
Epoch [1/500], Train Loss: 0.0004, Test Loss: 0.0003
Epoch: 1
Epoch [2/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 2
Epoch [3/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 3
Epoch [4/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 4
Epoch [5/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 5
Epoch [6/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 6
Epoch [7/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 7
Epoch [8/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 8
Epoch [9/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 9
Epoch [10/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 10
Epoch [11/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 11
Epoch [12/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 12
Epoch [13/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 13
Epoch [14/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 14
Epoch [15/500], Train Loss: 0.0003, Test Loss: 0.0003
Epoch: 15
Epoch [16/500], Train Loss: 0.0003, Test Loss: 0.

In [53]:
best_test_loss

0.00030753854934971345

In [54]:
metrics_df = pd.DataFrame({'Metric': ['Training RMSE', 'Test RMSE'],
'Value': [ConstantUnScaler(train_loss, min_gold = min_gold, max_gold=max_gold), ConstantUnScaler(test_loss, min_gold = min_gold, max_gold=max_gold)]})
#Save the metrics to a CSV file
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/lstm_no_embed")
metrics_file = "1step_1horizon_RMSE_1_metrics.csv"
metrics_df.to_csv(metrics_file, index=False)
print(f"Metrics saved to {metrics_file}\n {metrics_df}")
torch.save(best_model_state, '1step_1horizon_RMSE_1.pth')

Metrics saved to 1step_1horizon_RMSE_1_metrics.csv
           Metric  Value
0  Training RMSE   35.0
1      Test RMSE   35.0


10Step_5Horizon_RMSE_1Embed: Train Loss:    Test Loss: